# Filtering

In this notebooks we will explore the effect of filtering resulting motion vectors.

When running the optical flow algorithm to estimate displacement, we find the displacement at each pixel in the image. Some pixels are located in regions without cells we will therefore have very low displacements. However, noise and drift in the images can still give artificial velocity vectors in these pixels.

Displacement vectors that does not contains cells should therefore be filtered out, and we can do so by creating a mask that contains pixels with a significant high displacement.

Let us load the sample dataset and show what we mean

In [ ]:
import dataset
import mps_motion
import matplotlib.pyplot as plt

full_resolution = False
data = dataset.load_sample_data(full_resolution=full_resolution)

Now, let us rus the optical flow algorithm to find the displacements

In [ ]:
opt_flow = mps_motion.OpticalFlow(data, flow_algorithm="farneback")
u = opt_flow.get_displacements()

Now let ut compute the maximum displacement norm for each pixel

In [ ]:
u_norm_max = u.norm().max().compute()

In [ ]:
fig, ax = plt.subplots(1, 2)
im = ax[0].imshow(u_norm_max, cmap="hot")
fig.colorbar(im, ax=ax[0])
ax[1].imshow(data.frames[:, :, 0], cmap="gray")
plt.show()

As we can see, some regions without cells have very low displacements. This also includes that pilars.

Now let us zoom in at the lower left corner of the image, where we have the fenestrations. These pixels does not contain any cells so these pixels should have zero displacement

In [ ]:
fig, ax = plt.subplots(1, 2)
im = ax[0].imshow(u_norm_max[500:, :50], cmap="hot", vmin=0, vmax=0.25)
fig.colorbar(im, ax=ax[0])
ax[1].imshow(data.frames[500:, :50, 0], cmap="gray")
plt.show()

However, we see that displacement are not zero, but slightly positive. For reference let us also compute the average trace over time

In [ ]:
u_norm_mean = u.norm().mean().compute()
fig, ax = plt.subplots()
ax.plot(data.time_stamps, u_norm_mean)
plt.show()

Now let ut do the same with the velocity

In [ ]:
# We multiply be 1000 to convert from um/ms to um/s
v = opt_flow.get_velocities() * 1000
v_norm_max = v.norm().max().compute()

In [ ]:
fig, ax = plt.subplots(1, 2)
im = ax[0].imshow(v_norm_max, cmap="hot")
fig.colorbar(im, ax=ax[0])
ax[1].imshow(data.frames[:, :, 0], cmap="gray")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2)
im = ax[0].imshow(v_norm_max[500:, :50], cmap="hot", vmin=0, vmax=10)
fig.colorbar(im, ax=ax[0])
ax[1].imshow(data.frames[500:, :50, 0], cmap="gray")
plt.show()

And we can see that this effect is amplified in the velocities. Let us compute the average trace over time

In [ ]:
v_norm_mean = v.norm().mean().compute()
fig, ax = plt.subplots()
ax.plot(data.time_stamps[:-1], v_norm_mean)
plt.show()

What we can do, is to create a mask where pixels with low displacements are excluded. For simplicity let us exclude all pixels that have a lower value than the mean displacements

In [ ]:
mask = u_norm_max < u_norm_max.mean()

fig, ax = plt.subplots()
ax.imshow(data.frames[:, :, 0], cmap="gray")
ax.imshow(mask, alpha=0.5)
plt.show()


Now let us apply this mask to `u` and `v` and repeat the process

In [ ]:
u.apply_mask(mask)
v.apply_mask(mask)

In [ ]:
u_filtered_norm_max = u.norm().max().compute()

fig, ax = plt.subplots(2, 2)
im = ax[0, 0].imshow(u_filtered_norm_max, cmap="hot")
fig.colorbar(im, ax=ax[0, 0])
ax[0, 1].imshow(data.frames[:, :, 0], cmap="gray")
im = ax[1, 0].imshow(u_filtered_norm_max[500:, :50], cmap="hot", vmin=0, vmax=0.25)
fig.colorbar(im, ax=ax[1, 0])
ax[1, 1].imshow(data.frames[500:, :50, 0], cmap="gray")
plt.show()

In [ ]:
u_filtered_norm_mean = u.norm().mean().compute()
fig, ax = plt.subplots()
ax.plot(data.time_stamps, u_norm_mean, label="Original")
ax.plot(data.time_stamps, u_filtered_norm_mean, label="Filtered")
ax.legend()
plt.show()

And for the velocity 

In [ ]:
v_filtered_norm_max = v.norm().max().compute()

fig, ax = plt.subplots(2, 2)
im = ax[0, 0].imshow(v_filtered_norm_max, cmap="hot")
fig.colorbar(im, ax=ax[0, 0])
ax[0, 1].imshow(data.frames[:, :, 0], cmap="gray")
im = ax[1, 0].imshow(v_filtered_norm_max[500:, :50], cmap="hot", vmin=0, vmax=15)
fig.colorbar(im, ax=ax[1, 0])
ax[1, 1].imshow(data.frames[500:, :50, 0], cmap="gray")
plt.show()

In [ ]:
v_filtered_norm_mean = v.norm().mean().compute()
fig, ax = plt.subplots()
ax.plot(data.time_stamps[:-1], v_norm_mean, label="Original")
ax.plot(data.time_stamps[:-1], v_filtered_norm_mean, label="Filtered")
ax.legend()
plt.show()